In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import random
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
random.seed(23)

In [4]:
def generate_rand_matrix(rows, cols, p_ones):

    return np.random.choice([0, 1], size=(rows, cols), p=[1 - p_ones, p_ones])

def get_neighbors(mat):
    neighbor_dict = defaultdict(list)
    n,m =mat.shape[0],mat.shape[1]
    for i in range(n):
        for j in range(m):
            x = [[i+y[0], j+y[1], 0] for y in itertools.product([1,0,-1], [1,0,-1])]
            z = [e for e in x if (0 <= e[0] < n and 0 <= e[1] < m and e[:2] != [i, j])]
            neighbor_dict[(i, j)] = z

    for k,v in neighbor_dict.items():
        for e in v:
            if mat[e[0],e[1]]==1:
                e[2]=1
            else:
                e[2]=0

    return neighbor_dict


def next_gen(mat):
    neighbor_sum_dict = defaultdict(int)
    mat_copy = mat.copy()
    neighbors = get_neighbors(mat)

    for k,v in neighbors.items():
        neighbors_sum = sum([e[2] for e in v])
        
        neighbor_sum_dict[k] = neighbors_sum
        
    for k,v in neighbor_sum_dict.items():
        i, j = k

        if (v < 2 or v > 3) and mat[i, j] == 1:
            mat_copy[i, j] = 0

        elif mat[i, j] == 0 and v == 3:
            mat_copy[i, j] = 1
            
    return mat_copy



def indvidual_matrix_data_collection(mat,gen_num,run_num,collection_dict,alive_percent):

    collection_dict["gen_dict"]["alive_cells"].append(int(mat.sum()))
    collection_dict["gen_dict"]["gen"].append(gen_num)
    collection_dict["gen_dict"]["run"].append(run_num)
    collection_dict["gen_dict"]["initial_percent_alive"].append(alive_percent)


    return collection_dict

     

In [ ]:


def run_game(mat,run_num,master_data_collection_dict,alive_percent,max_steps=1000):

    indvidual_matrix_data_collection(mat,0,run_num,master_data_collection_dict,alive_percent)

    step = 1
    steady_count = 0  

    while step < max_steps:
        new_mat = next_gen(mat)
        indvidual_matrix_data_collection(new_mat,step,run_num,master_data_collection_dict,alive_percent)

        if np.array_equal(new_mat, mat):
            steady_count += 1
        else:
            steady_count = 0 

        if steady_count == 2:
            master_data_collection_dict["run_dict"]["term_reason"].append("steady")
            master_data_collection_dict["run_dict"]["last_gen"].append(step)
            master_data_collection_dict["run_dict"]["run"].append(run_num)
            master_data_collection_dict["run_dict"]["initial_percent_alive"].append(alive_percent)
            break
        
        if np.all(new_mat == 0):
            master_data_collection_dict["run_dict"]["term_reason"].append("dead")
            master_data_collection_dict["run_dict"]["last_gen"].append(step)
            master_data_collection_dict["run_dict"]["run"].append(run_num)
            master_data_collection_dict["run_dict"]["initial_percent_alive"].append(alive_percent)
            break
        
        mat = new_mat  
        step += 1
        
    if step == max_steps:
        master_data_collection_dict["run_dict"]["term_reason"].append("max")
        master_data_collection_dict["run_dict"]["last_gen"].append(step)
        master_data_collection_dict["run_dict"]["run"].append(run_num)
        master_data_collection_dict["run_dict"]["initial_percent_alive"].append(alive_percent)
     

In [10]:
result_dict = defaultdict(lambda: defaultdict(list))

In [11]:
for i in np.arange(.05, 1, 0.05):
    p = round(i, 2)  
    for x in range(3):  
        test_matrix = generate_rand_matrix(100, 100, p)
        run_game(test_matrix, x, result_dict, p)

output_dict = {k:pd.DataFrame(v) for k,v in result_dict.items()}
generation_df = output_dict["gen_dict"]
run_df = output_dict["run_dict"]

KeyboardInterrupt: 

In [ ]:
merged_df = pd.merge(run_df,generation_df,on=["run","initial_percent_alive"],how="left")

In [ ]:
merged_df.to_csv(r"C:\Users\Will\Documents\school_data\gol.csv")